In [6]:
%cd /home/tkm290/repos/erc

/home/tkm290/repos/erc


In [7]:
from sklearn.metrics import f1_score
from tqdm.tqdm import tqdm
import os
from glob import glob
import argparse
from fairseq.models.roberta import RobertaModel
import sys
DATASET_DIR = "Datasets/"
DATASET = 'MELD'
use_cuda = True

if DATASET not in ['MELD', 'IEMOCAP', 'AFEW', 'CAER']:
    sys.exit(f"{DATASET} is not supported!")

roberta = RobertaModel.from_pretrained(
    'models/roberta.base',
    checkpoint_file=f'{DATASET}.pt',
    data_name_or_path=os.path.join(DATASET_DIR, DATASET, 'roberta/bin')
)

def label_fn(label): return roberta.task.label_dictionary.string(
    [label + roberta.task.label_dictionary.nspecial]
)

roberta.eval()  # disable dropout
if use_cuda:
    roberta.cuda()

y_true = {}
y_pred = {}

for SPLIT in tqdm(['train', 'val', 'test']):

    y_true[SPLIT] = []
    y_pred[SPLIT] = []

    X = os.path.join(DATASET_DIR, DATASET, 'roberta', SPLIT + '.input0')
    with open(X, 'r') as stream:
        X = [line.strip() for line in stream.readlines()]

    Y = os.path.join(DATASET_DIR, DATASET, 'roberta', SPLIT + '.label')
    with open(Y, 'r') as stream:
        Y = [line.strip() for line in stream.readlines()]

    for line, label in tqdm(zip(X, Y)):
        tokens = roberta.encode(line)
        pred = label_fn(roberta.predict(
            DATASET + '_head', tokens).argmax().item())

        y_true[SPLIT].append(label)
        y_pred[SPLIT].append(pred)

for SPLIT in ['train', 'val', 'test']:
    score = f1_score(y_true[SPLIT], y_pred[SPLIT], average='weighted')
    print(f"Weighed f1 score on {DATASET}, {SPLIT}: {score}")

  0%|          | 0/3 [00:00<?, ?it/s]
0it [00:00, ?it/s]
9it [00:00, 85.39it/s]
18it [00:00, 82.99it/s]
27it [00:00, 84.12it/s]
36it [00:00, 84.98it/s]
45it [00:00, 85.12it/s]
54it [00:00, 85.39it/s]
63it [00:00, 85.38it/s]
72it [00:00, 85.58it/s]
81it [00:00, 85.36it/s]
90it [00:01, 85.40it/s]
99it [00:01, 84.84it/s]
108it [00:01, 84.64it/s]
117it [00:01, 84.56it/s]
126it [00:01, 84.52it/s]
135it [00:01, 84.67it/s]
144it [00:01, 84.71it/s]
153it [00:01, 84.75it/s]
162it [00:01, 84.93it/s]
171it [00:02, 85.23it/s]
180it [00:02, 85.52it/s]
189it [00:02, 85.65it/s]
198it [00:02, 85.01it/s]
207it [00:02, 84.95it/s]
216it [00:02, 84.79it/s]
225it [00:02, 84.97it/s]
234it [00:02, 84.23it/s]
248it [00:02, 84.41it/s]
  0%|          | 0/3 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [9]:
roberta.

In [10]:
roberta.fill_mask('The first Star wars movie came out in <mask>', topk=3)

AttributeError: 'SentencePredictionTask' object has no attribute 'mask_idx'

In [12]:
roberta.